# Color Filtering using HSV
In the make_blue challenge, we were operating in the BGR "colorspace" (Blue, Green, Red), but there are other colorspaces -- including HSV (hue, saturation, value).  HSV is also three values, but they operate differently.

You can get an idea by looking at this diagram:

![HSV Color wheel](https://upload.wikimedia.org/wikipedia/commons/0/0d/HSV_color_solid_cylinder_alpha_lowgamma.png)

The first step in leveraging HSV is to convert the image into the HSV colorspace...  Let's use our favorite image and convert to to HSV and see what it looks like.  We'll load up the image and display it standard first.

In [4]:
import cv2
import numpy as np
import os

def load_image_file(image_file):        # I used a function here to combine the test with the load
    if(os.path.isfile(image_file)):     # os.path.isfile returns true if the file exists, why is this needed?
	    img = cv2.imread(image_file)
    else:
	    print("Invalid file, exiting....")
	    exit(1)
    return(img)

team_image_file = "..\..\images\Jagbots2017.jpg"

team_image_BGR = load_image_file(team_image_file)

cv2.imshow("Image", team_image_BGR)                          # this function will open a window with the image
cv2.waitKey(0)                                    # this function will wait until a key is pressed
    
cv2.destroyAllWindows()


So we've loaded it up, now let's convert it to the HSV colorspace.  This next snippet requires that the above code has been run, so be sure you ran it!

We're going to create a new image in the HSV colorspace.  This conversion is similar to converting to Grayscale, but it produces an image that is color, but not the same colors...

In [5]:
# convert from BGR to HSV
#
team_image_HSV = cv2.cvtColor(team_image_BGR, cv2.COLOR_BGR2HSV)

cv2.imshow("HSV Image", team_image_HSV)
cv2.waitKey(0)

cv2.destroyAllWindows()

If it all worked correctly, you're seeing a pretty different image -- like one that was taken using a nuclear camera flash.

Obviously we are not doing this to make it prettier to human eyes -- so why bother?  We convert to HSV for the same reason we will often convert to Grayscale, it makes it easier to perform other operations -- like color filtering.

Since we are going to be chasing the reflective tape used by FIRST, let's switch to a sample image from last season...

In [6]:
import cv2
import numpy as np
import os

def load_image_file(image_file):        # I used a function here to combine the test with the load
    if(os.path.isfile(image_file)):     # os.path.isfile returns true if the file exists, why is this needed?
	    img = cv2.imread(image_file)
    else:
	    print("Invalid file, exiting....")
	    exit(1)
    return(img)

image_file = "../../images/tape_sample_images/sample7.png"

image_BGR = load_image_file(image_file)

cv2.imshow("Image", image_BGR)                  # this function will open a window with the image
cv2.waitKey(0)                                  # this function will wait until a key is pressed
    

# convert from BGR to HSV
#
image_HSV = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2HSV)

cv2.imshow("HSV Image", image_HSV)
cv2.waitKey(0)

cv2.destroyAllWindows()

Looking at these images, you get an idea of why that tape is so special -- it really stands out.  Now let's try to filter for that color...

In [7]:
# define range of green color in HSV
lower_tape = np.array([50,90,110])
upper_tape = np.array([100,255,255])

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(image_HSV, lower_tape, upper_tape)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(image_BGR,image_BGR, mask= mask)

cv2.imshow('image',image_BGR)
cv2.imshow('mask',mask)
cv2.imshow('res',res)

# Let's define our kernel size
kernel = np.ones((5,5), np.uint8)

# Now we erode
erosion = cv2.erode(res, kernel, iterations = 1)
cv2.imshow('Erosion', erosion)

cv2.waitKey(0)

cv2.destroyAllWindows()

Let's find the rectangles

In [18]:
# Grayscale
gray = cv2.cvtColor(erosion,cv2.COLOR_BGR2GRAY)

# Find Canny edges
edged = cv2.Canny(gray, 30, 200)
cv2.imshow('Canny Edges', edged)
cv2.waitKey(0)

# Finding Contours
# Use a copy of your image e.g. edged.copy(), since findContours alters the image
_, contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(contours)
cv2.imshow('Canny Edges After Contouring', edged)
cv2.waitKey(0)

print("Number of Contours found = " + str(len(contours)))

# Draw all contours
# Use '-1' as the 3rd parameter to draw all
cv2.drawContours(image_BGR, contours, -1, (255,0,0), 3)
cv2.imshow('Contours', image_BGR)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[576, 470]],

       [[576, 484]],

       [[575, 485]],

       [[575, 509]],

       [[574, 510]],

       [[574, 512]],

       [[575, 513]],

       [[575, 523]],

       [[574, 524]],

       [[574, 530]],

       [[575, 531]],

       [[587, 531]],

       [[588, 532]],

       [[595, 532]],

       [[595, 522]],

       [[596, 521]],

       [[596, 518]],

       [[597, 517]],

       [[597, 482]],

       [[598, 481]],

       [[598, 477]],

       [[597, 476]],

       [[597, 474]],

       [[598, 473]],

       [[598, 471]],

       [[586, 471]],

       [[585, 470]]], dtype=int32), array([[[473, 468]],

       [[472, 469]],

       [[467, 469]],

       [[467, 472]],

       [[466, 473]],

       [[466, 531]],

       [[470, 531]],

       [[471, 532]],

       [[486, 532]],

       [[487, 531]],

       [[487, 501]],

       [[488, 500]],

       [[488, 499]],

       [[489, 498]],

       [[489, 470]],

       [[488, 469]],

       [[476, 469]],

       [[475, 468

In [9]:
# print(contours[0][0])
print(np.amin(contours[0],0))
print(np.amax(contours[0],0))
print(np.amin(contours[1],0))
print(np.amax(contours[1],0))
# print(contours[0][-1])
# print(contours[0])
# let's verify
contourOne = contours[0]
print(np.amin(contourOne[0]))
print(np.amax(contourOne[0]))
print(np.amin(contourOne[1]))
print(np.amax(contourOne[1]))

print(contourOne)
dir(contours)

[[574 470]]
[[598 532]]
[[466 468]]
[[489 532]]
470
576
484
576
[[[576 470]]

 [[576 484]]

 [[575 485]]

 [[575 509]]

 [[574 510]]

 [[574 512]]

 [[575 513]]

 [[575 523]]

 [[574 524]]

 [[574 530]]

 [[575 531]]

 [[587 531]]

 [[588 532]]

 [[595 532]]

 [[595 522]]

 [[596 521]]

 [[596 518]]

 [[597 517]]

 [[597 482]]

 [[598 481]]

 [[598 477]]

 [[597 476]]

 [[597 474]]

 [[598 473]]

 [[598 471]]

 [[586 471]]

 [[585 470]]]


['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

Using the maximum and minimum coordinates of the contours found, we can draw two rectangles around the glow tape with straight edges.

In [20]:
print(np.amin(contours[0],0))
print(np.amax(contours[0],0))
corner1 = tuple(np.amin(contours[0],0)[0])
corner2 = tuple(np.amax(contours[0],0)[0])
print(corner1)
print(corner2)
corner3 = tuple(np.amin(contours[1],0)[0])
corner4 = tuple(np.amax(contours[1],0)[0])
final1 = cv2.rectangle(image_BGR,corner1,corner2,(0,255,0),3)
final2 = cv2.rectangle(image_BGR,corner3,corner4,(0,255,0),3)
cv2.imshow('Result', final1 + final2)
cv2.waitKey(0)
    
cv2.destroyAllWindows()

[[574 470]]
[[598 532]]
(574, 470)
(598, 532)
